# README

20240531: 作成 (Tomoro Yanase)

解析ライブラリmod_scaleの含まれる各機能を説明する.
※ 階層を整理していく上での命名規則については自信ないので必要に応じて改めたい(ライブラリ・パッケージ・モジュール, など)


## g_file.py: ファイル読込関係
* get_fpathlist: ファイルパスリストの取得
  * 引数: 
      * dir1: データのディレクトリ名
      * ftype: ファイルタイプ名(history, topo, ...)
      * domainlabel: ドメインラベル
      * timelabel: 時間ラベル
      * PRC_NUM_X: X方向の分割ファイル数(実行時プロセス数)
      * PRC_NUM_Y: Y方向の分割ファイル数(実行時プロセス数)
  * 戻り値:
      * ファイルパスがネストされたリスト

* get_xrvar: xarrayオブジェクトの取得
  * 引数:
      * ファイルパスがネストされたリスト
  * 戻り値:
      * 結合されたxarrayオブジェクト


## g_mpi.py: MPI並列関係
* get_mpi: MPIプロセスの立ち上げ
  * 引数:
      * なし
  * 戻り値:
      * comm: MPIオブジェクト
      * size: MPIサイズ
      * rank: MPIランク

* get_and_check_prc:  MPI設定の整合性チェックと解析MPIプロセス一つあたりの担当分割ファイル数(実行時プロセス数)の取得
  * 引数: 
      * PRC_NUM_X: X方向の分割ファイル数(実行時プロセス数)
      * PRC_NUM_Y: Y方向の分割ファイル数(実行時プロセス数)
      * PRC_NUM_X_ANL: X方向の解析プロセス数
      * PRC_NUM_Y_ANL: Y方向の解析プロセス数
      * size: 総解析プロセス数
  * 戻り値:
      * PRC_NUM_X_PER_ANL: 解析プロセス一つあたりのX方向分割ファイル数(実行時プロセス数)
      * PRC_NUM_Y_PER_ANL: 解析プロセス一つあたりのY方向分割ファイル数(実行時プロセス数)

* get_fpathlist_mpi: 解析MPIプロセスが担当するファイルパスリストの取得
    * 引数:
        * データのディレクトリ名: 
        * ファイルタイプ名(history, topo, ...): 
        * ドメインラベル:
        * 時間ラベル: 
        * PRC_NUM_X: X方向の分割ファイル数(実行時プロセス数)
        * PRC_NUM_Y: Y方向の分割ファイル数(実行時プロセス数)
        * PRC_NUM_X_ANL: X方向の解析プロセス数
        * PRC_NUM_Y_ANL: Y方向の解析プロセス数
        * size: MPIサイズ
        * rank: MPIランク
    * 戻り値:
        * fpathlist

* check_prcnum_mpi: チェック
    * 引数:
        * PRC_NUM_X
        * PRC_NUM_Y
        * PRC_NUM_X_ANL
        * PRC_NUM_Y_ANL
        * size
        * rank
    * 戻り値
        * 

* combine_var2d_blockavg: 子プロセス内でブロック平均した二次元変数を, 親プロセスで受け取り結合
    * 引数:
        * sendbuf: 子プロセスが送る, ブロック平均した二次元配列
        * recvbuf: 子プロセスから送られてきた, ブロック平均した二次元変数を, 親プロセスで格納する二次元配列
        * PRC_NUM_Y_ANL:
        * PRC_NUM_X_ANL: 
    * 戻り値:
        * var2d_avg_combined: 結合された二次元配列
    * 例:
            xrvar = get_xrvar(fpathlist)
            var2d_avg = get_blcavg(xrvar,varname1,blct,blcy,blcx)
            sendbuf = np.array(var2d_avg,"d")
            recvbuf = None
            if rank == 0:
                recvbuf = np.array(np.tile(np.empty_like(sendbuf),(size,1,1,1)), dtype='d')  
            comm.Gather(sendbuf, recvbuf, root=0)
            if rank == 0:
                var2d_avg_combined = combine_var2d_blockavg(sendbuf,recvbuf,PRC_NUM_Y_ANL,PRC_NUM_X_ANL)


## g_block_sort.py: ブロック平均・ソート処理関係
* get_key2d: 水平方向にブロック平均した二次元変数をキーにソートした(並べ替えた)インデックスの配列を取得
     * 引数:
         * xrvar: xarrayオプジェクト
         * key2dname: キーとする二次元変数名(e.g., MSLP, PW)
         * blct,blcy,blcx: ブロック平均の時空間幅(時間方向, y方向, x方向)
     * 戻り値:
         * index: インデックス. ブロック平均した大気カラムを横一列に並べてnumpy.argsortした結果 https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.argsort.html
         * key2d0: 
         * weights_sum_sorted:  ブロック間で平均する際のブロックの重み(e.g., cell_area)

* get_var2d_sorted_bykey2d
    * 引数
        * xrvar
        * var2dname
        * blct
        * blcy
        * blcx
        * index: ソートのインデックス
    * 戻り値:
        *  var2d_sorted: ソートされた二次元変数

* get_var3d_sorted_bykey2d
    * 引数:
        * xrvar
        * var3dname
        * blct,blcy,blcx
        * index
    * 戻り値:
        * var3d_sorted: ソートされた三次元変数

* get_blcavg
     * 引数:
         * xrvar
         * var2dname
         * blct,blcy,blcx
     * 戻り値:
         * var2d_avg: ブロック平均された二次元変数
 
